In [1]:
pip install Levenshtein

In [2]:
pip install torch

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
pip install transformers[torch]

In [5]:
import pandas as pd


In [6]:
pip install scikit-learn

In [7]:
df = pd.read_csv('/content/drive/MyDrive/dataset.csv', encoding='ISO-8859-1')
df.shape

half_size = len(df) // 5
trimmed_df = df.head(half_size)
trimmed_df.shape
df=trimmed_df

In [8]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.10, shuffle=True)
train_df.shape, test_df.shape

((180000, 2), (20000, 2))

In [9]:
half_size = len(test_df) // 20
trimmed_df = test_df.head(half_size)
trimmed_df.shape
df=trimmed_df
df.shape

(1000, 2)

In [10]:
pip install sentencepiece


In [11]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
model_name = '/content/drive/MyDrive/t5base-finetuned'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
df.describe

<bound method NDFrame.describe of                                                     input  \
132487  WGU Online University, Salt Lake City, UT. | O...   
579     Thank you, Rando! Iâm always happy to find c...   
110657  Can't do a screen shot a tthe moment as sit in...   
30730   The second ad, called â765 Press Releases,â...   
102551  Whilst the improvement in her Psoriasis is gre...   
...                                                   ...   
105829  A good variety in a diferent way to come to sh...   
69014   If you see photos of the fun that were during ...   
26939   James has playing in 237 career playoff with t...   
63953   Comprising of paneled bath with shower attachm...   
79500   A 12.5m wide x 37m Haven courtyard cover canop...   

                                                   output  
132487  WGU Online University, Salt Lake City, UT. | O...  
579     Thank you, Rando! Iâm always happy to find c...  
110657  Can't do a screen shot at the moment as sat i

In [13]:
def correct_grammar(input_text,num_return_sequences=1):
  device = torch.device("cuda" if torch.cuda.is_available() else "cuda")
  batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text[0]

In [14]:
import pandas as pd

# Create a new DataFrame by applying the function to each row of the original DataFrame 'df'
new_df = df.copy()  # Copy the original DataFrame to a new one

# Define the function to correct grammar
def correct_grammar_for_row(row):
    row['output_predicted'] = correct_grammar(row['input'], num_return_sequences=1)
    return row

# Apply the function to the new DataFrame
new_df = new_df.apply(correct_grammar_for_row, axis=1)

# Print the new DataFrame
print(new_df)


                                                    input  \
132487  WGU Online University, Salt Lake City, UT. | O...   
579     Thank you, Rando! Iâm always happy to find c...   
110657  Can't do a screen shot a tthe moment as sit in...   
30730   The second ad, called â765 Press Releases,â...   
102551  Whilst the improvement in her Psoriasis is gre...   
...                                                   ...   
105829  A good variety in a diferent way to come to sh...   
69014   If you see photos of the fun that were during ...   
26939   James has playing in 237 career playoff with t...   
63953   Comprising of paneled bath with shower attachm...   
79500   A 12.5m wide x 37m Haven courtyard cover canop...   

                                                   output  \
132487  WGU Online University, Salt Lake City, UT. | O...   
579     Thank you, Rando! Iâm always happy to find c...   
110657  Can't do a screen shot at the moment as sat in...   
30730   The second ad, 

In [18]:
df.to_csv('datafinal-t5base.csv', index=False)  # Set index=False to exclude the index column


In [15]:
import pandas as pd
import Levenshtein



# Function to calculate Levenshtein distance and similarity score
def calculate_similarity(row):
    original_output = row["output"]
    predicted_output = row["output_predicted"]

    # Calculate the Levenshtein distance
    distance = Levenshtein.distance(original_output, predicted_output)

    # Calculate a similarity score (normalized Levenshtein similarity)
    max_len = max(len(original_output), len(predicted_output))
    similarity = 1 - (distance / max_len)

    return similarity

# Apply the calculate_similarity function to each row and store the result in a new column
new_df['similarity_score'] = new_df.apply(calculate_similarity, axis=1)

# Print the DataFrame with the similarity scores
print(new_df)

                                                    input  \
132487  WGU Online University, Salt Lake City, UT. | O...   
579     Thank you, Rando! Iâm always happy to find c...   
110657  Can't do a screen shot a tthe moment as sit in...   
30730   The second ad, called â765 Press Releases,â...   
102551  Whilst the improvement in her Psoriasis is gre...   
...                                                   ...   
105829  A good variety in a diferent way to come to sh...   
69014   If you see photos of the fun that were during ...   
26939   James has playing in 237 career playoff with t...   
63953   Comprising of paneled bath with shower attachm...   
79500   A 12.5m wide x 37m Haven courtyard cover canop...   

                                                   output  \
132487  WGU Online University, Salt Lake City, UT. | O...   
579     Thank you, Rando! Iâm always happy to find c...   
110657  Can't do a screen shot at the moment as sat in...   
30730   The second ad, 

In [16]:
average_similarity = new_df['similarity_score'].mean()
print(average_similarity)

0.8541912195931857
